In [1]:
# ----------------------------
# 📦 Imports and Setup
# ----------------------------
import pandas as pd
import datetime
from dash import Dash, html, dcc, Output, Input
import dash_leaflet as dl
import dash_leaflet.express as dlx


In [2]:
# ----------------------------
# 📊 Load and Prepare Data
# ----------------------------
df = pd.read_excel("cw_perform_event_df.xlsx")

# Filter to valid vehicle types and valid coordinates
df = df[df["vehicleType"].isin(["FL", "ARC"])]
df = df.dropna(subset=["completionLatitude", "completionLongitude"])

# Parse dates and extract date column
df["start"] = pd.to_datetime(df["start"])
df["end"] = pd.to_datetime(df["end"])
df["date"] = df["start"].dt.date

# Create coordinates column
df["coord"] = list(zip(df["completionLatitude"], df["completionLongitude"]))

# Clean up tourNo: if NaN, assign dummy value so it doesn’t break UI
df["tourNo"] = df["tourNo"].fillna("Unassigned")


In [3]:
# ----------------------------
# 🚀 Dash App Layout
# ----------------------------
app = Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H2("🚛 Truck Tour Viewer"),
    html.Div([
        html.Label("Select Vehicle Type:"),
        dcc.Dropdown(
            id="type-dropdown",
            options=[{"label": t, "value": t} for t in sorted(df["vehicleType"].unique())],
            value="FL"
        ),
        html.Label("Select Date:"),
        dcc.Dropdown(id="date-dropdown"),
        html.Label("Select Truck ID:"),
        dcc.Dropdown(id="truck-dropdown"),
        html.Label("Select Tour ID:"),
        dcc.Dropdown(id="tour-dropdown")
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"}),

    html.Div([
        dl.Map(
            id="map",
            center=(48.7656, 11.4237),
            zoom=11,
            style={"width": "100%", "height": "700px"},
            children=[dl.TileLayer()]
        )
    ], style={"width": "68%", "display": "inline-block", "paddingLeft": "2%"})
])


In [4]:
# ----------------------------
# 🔄 Update Date Dropdown
# ----------------------------
@app.callback(
    Output("date-dropdown", "options"),
    Input("type-dropdown", "value")
)
def update_date_dropdown(vehicle_type):
    dates = df[df["vehicleType"] == vehicle_type]["date"].unique()
    return [{"label": str(date), "value": str(date)} for date in sorted(dates)]


In [5]:
# ----------------------------
# 🔄 Update Truck Dropdown
# ----------------------------
@app.callback(
    Output("truck-dropdown", "options"),
    [Input("type-dropdown", "value"), Input("date-dropdown", "value")]
)
def update_truck_dropdown(vehicle_type, selected_date):
    if not selected_date:
        return []
    date_obj = datetime.datetime.strptime(selected_date, "%Y-%m-%d").date()
    trucks = df[(df["vehicleType"] == vehicle_type) & (df["date"] == date_obj)]["truck"].unique()
    return [{"label": truck, "value": truck} for truck in sorted(trucks)]


In [6]:
# ----------------------------
# 🔄 Update Tour Dropdown
# ----------------------------
@app.callback(
    Output("tour-dropdown", "options"),
    [Input("type-dropdown", "value"), Input("date-dropdown", "value"), Input("truck-dropdown", "value")]
)
def update_tour_dropdown(vehicle_type, selected_date, truck_id):
    if not selected_date or not truck_id:
        return []
    date_obj = datetime.datetime.strptime(selected_date, "%Y-%m-%d").date()
    tours = df[
        (df["vehicleType"] == vehicle_type) &
        (df["date"] == date_obj) &
        (df["truck"] == truck_id)
    ]["tourNo"].unique()
    return [{"label": str(t), "value": str(t)} for t in sorted(tours)]


In [7]:
# ----------------------------
# 🗺️ Map Callback
# ----------------------------
@app.callback(
    Output("map", "children"),
    [Input("type-dropdown", "value"),
     Input("date-dropdown", "value"),
     Input("truck-dropdown", "value"),
     Input("tour-dropdown", "value")]
)
def update_map(vehicle_type, selected_date, truck_id, tour_no):
    if not selected_date or not truck_id or not tour_no:
        return [dl.TileLayer()]

    df_filtered = df[
        (df["vehicleType"] == vehicle_type) &
        (df["date"] == datetime.datetime.strptime(selected_date, "%Y-%m-%d").date()) &
        (df["truck"] == truck_id) &
        (df["tourNo"].astype(str) == str(tour_no))
    ].sort_values("start")

    if df_filtered.empty:
        return [dl.TileLayer()]

    coords = df_filtered["coord"].tolist()
    layers = [dl.TileLayer()]
    markers = []

    for i, row in df_filtered.iterrows():
        coord = row["coord"]
        popup = dl.Popup([
            html.B(f"Truck: {row['truck']}"), html.Br(),
            f"Client: {row['clientAddress']}", html.Br(),
            f"Start: {row['start']}", html.Br(),
            f"End: {row['end']}", html.Br(),
            f"CO₂: {row.get('event_co2Emission', 'N/A')}", html.Br(),
            f"Fuel Rate: {row.get('event_fuel_rate', 'N/A')}", html.Br(),
            f"Speed: {row.get('wheel_speed', 'N/A')}"
        ], maxWidth=300)

        if i == df_filtered.index[0]:
            color = "green"
        elif i == df_filtered.index[-1]:
            color = "red"
        else:
            color = "blue"

        markers.append(dl.CircleMarker(center=coord, radius=6, color=color, fill=True, fillOpacity=0.9, children=[popup]))

    layers.extend(markers)
    layers.append(dl.Polyline(positions=coords, color="black", weight=3))

    # Add legend
    legend = dlx.categorical_colorbar(
        categories=["Start", "Intermediate", "End"],
        colorscale=["green", "blue", "red"],
        width=300,
        height=50,
        position="bottomright"
    )
    layers.append(legend)

    return layers

In [8]:
# ----------------------------
# 🚀 Run App
# ----------------------------
if __name__ == '__main__':
    app.run_server(debug=True)